## [Analyzing structured data](https://python.langchain.com/docs/use_cases/tabular)


In [1]:
import langchain
langchain.__version__

'0.0.227'

In [2]:
import os

oai_key = open('../_secret/yuki-openaikey-20230613.txt', 'r').read()
os.environ["OPENAI_API_KEY"] = oai_key
#os.environ["OPENAI_API_KEY"]

In [5]:
import pandas as pd

df = pd.read_csv('../../data/BigClass.csv')
print(df.shape)
df.head(10)

(40, 5)


,Name,Age,Sex,Height_inch,Weight_pound
0,KATIE,12,F,59,95
1,LOUISE,12,F,61,123
2,JANE,12,F,55,74
3,JACLYN,12,F,66,145
4,LILLIE,12,F,52,64
5,TIM,12,M,60,84
6,JAMES,12,M,61,128
7,ROBERT,12,M,51,79
8,BARBARA,13,F,60,112
9,ALICE,13,F,61,107


### [CSV Agent](https://python.langchain.com/docs/modules/agents/toolkits/csv.html)

In [18]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

In [19]:
?create_csv_agent

In [20]:
agent = create_csv_agent(
    llm=OpenAI(temperature=0),
    path='../../data/BigClass.csv',
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [21]:
type(agent)

langchain.agents.agent.AgentExecutor

In [22]:
agent.run("how many rows are there?")



> Entering new  chain...
Thought: I need to count the rows
Action: python_repl_ast
Action Input: len(df)
Observation: 40
Thought: I now know the final answer
Final Answer: There are 40 rows.

> Finished chain.


'There are 40 rows.'

In [23]:
agent.run("行数と列数は？")



> Entering new  chain...
Thought: データフレームの行数と列数を確認する
Action: python_repl_ast
Action Input: df.shape
Observation: (40, 5)
Thought: 確認できた
Final Answer: データフレームの行数は40、列数は5です。

> Finished chain.


'データフレームの行数は40、列数は5です。'

In [24]:
agent.run("カラム名とそれぞれのデータ型は？")



> Entering new  chain...
Thought: データフレームを見てみる
Action: python_repl_ast
Action Input: df.dtypes
Observation: Name            object
Age              int64
Sex             object
Height_inch      int64
Weight_pound     int64
dtype: object
Thought: 答えを作る
Final Answer: カラム名はName, Age, Sex, Height_inch, Weight_poundで、それぞれのデータ型はobject, int64, object, int64, int64です。

> Finished chain.


'カラム名はName, Age, Sex, Height_inch, Weight_poundで、それぞれのデータ型はobject, int64, object, int64, int64です。'

In [25]:
agent.run("Height_inchはインチで表示されてます。センチメートル（cm）へ変換した値をHeight_cmカラムに格納してください。")



> Entering new  chain...
Thought: Height_inchをcmに変換する必要がある
Action: python_repl_ast
Action Input: df['Height_cm'] = df['Height_inch'] * 2.54
Observation: 
Thought: Height_cmカラムが追加された
Final Answer: df['Height_cm']にHeight_inchをcmに変換した値が格納されました。

> Finished chain.


"df['Height_cm']にHeight_inchをcmに変換した値が格納されました。"

In [26]:
agent.run("Weight_poundはポンドで表示されてます。キログラム（kg）へ変換した値をWeight_kgカラムに格納してください。")



> Entering new  chain...
Thought: Weight_poundをkgに変換する必要がある
Action: python_repl_ast
Action Input: df['Weight_kg'] = df['Weight_pound'] * 0.453592
Observation: 
Thought: Weight_kgカラムが追加された
Final Answer: df['Weight_kg']にWeight_poundをkgに変換した値が格納されました。

> Finished chain.


"df['Weight_kg']にWeight_poundをkgに変換した値が格納されました。"

In [27]:
agent.run("データを表示してください")



> Entering new  chain...
Thought: データフレームを表示する
Action: python_repl_ast
Action Input: print(df.head())
Observation:      Name  Age Sex  Height_inch  Weight_pound  Height_cm  Weight_kg
0   KATIE   12   F           59            95     149.86  43.091240
1  LOUISE   12   F           61           123     154.94  55.791816
2    JANE   12   F           55            74     139.70  33.565808
3  JACLYN   12   F           66           145     167.64  65.770840
4  LILLIE   12   F           52            64     132.08  29.029888

Thought: 答えが得られた
Final Answer: 上記のデータが表示されました。

> Finished chain.


'上記のデータが表示されました。'

In [28]:
agent.run("Height_cmとWeight_kgからBMIを計算しBMIカラムを作成してください")



> Entering new  chain...
Thought: BMIを計算するにはHeight_inchとWeight_poundを変換する必要がある
Action: python_repl_ast
Action Input: df['Height_cm'] = df['Height_inch'] * 2.54
Observation: 
Thought: Weight_poundをWeight_kgに変換する
Action: python_repl_ast
Action Input: df['Weight_kg'] = df['Weight_pound'] * 0.453592
Observation: 
Thought: BMIを計算する
Action: python_repl_ast
Action Input: df['BMI'] = df['Weight_kg'] / (df['Height_cm']/100)**2
Observation: 
Thought: I now know the final answer
Final Answer: BMIカラムを作成しました

> Finished chain.


'BMIカラムを作成しました'

In [29]:
agent.run("データをBigClass_v2.csvとして保存してください")



> Entering new  chain...
Thought: I need to save the dataframe as a csv file
Action: python_repl_ast
Action Input: df.to_csv('BigClass_v2.csv')
Observation: 
Thought: I now know the final answer
Final Answer: The dataframe has been saved as a csv file called BigClass_v2.csv.

> Finished chain.


'The dataframe has been saved as a csv file called BigClass_v2.csv.'

In [31]:
!ls

BigClass_v2.csv                        UseCases_AnalyzingStructuredData.ipynb
LangChain_Quickstart.ipynb


In [34]:
pd.read_csv('BigClass_v2.csv').head()

,Unnamed: 0,Name,Age,Sex,Height_inch,Weight_pound,Height_cm,Weight_kg,BMI
0,0,KATIE,12,F,59,95,149.86,43.091240,19.187462
1,1,LOUISE,12,F,61,123,154.94,55.791816,23.240389
2,2,JANE,12,F,55,74,139.70,33.565808,17.199043
3,3,JACLYN,12,F,66,145,167.64,65.770840,23.403353
4,4,LILLIE,12,F,52,64,132.08,29.029888,16.640687


In [46]:
agent.run("Age×Sexで集計を実施してください")



> Entering new  chain...
Thought: I need to group the data by Age and Sex
Action: python_repl_ast
Action Input: df.groupby(['Age', 'Sex']).size()
Observation: Age  Sex
12   F      5
     M      3
13   F      3
     M      4
14   F      5
     M      7
15   F      2
     M      5
16   F      2
     M      1
17   F      1
     M      2
dtype: int64
Thought: I now know the final answer
Final Answer: Age×Sexで集計した結果は、12歳の女性が5人、12歳の男性が3人、13歳の女性が3人、13歳の男性が4人、14歳の女性が5人、14歳の男性が7人、15歳の女性が2人、15歳の男性が5人、16歳の女性が2人、16歳の男性が1人、17歳の女性が1人、17歳の男性が2人である。

> Finished chain.


'Age×Sexで集計した結果は、12歳の女性が5人、12歳の男性が3人、13歳の女性が3人、13歳の男性が4人、14歳の女性が5人、14歳の男性が7人、15歳の女性が2人、15歳の男性が5人、16歳の女性が2人、16歳の男性が1人、17歳の女性が1人、17歳の男性が2人である。'

In [38]:
agent.run("Age×Sexの集計結果を表示し、その結果をBigClass_cross.csvとして保存してください")



> Entering new  chain...
Thought: I need to aggregate the Age and Sex columns and then save the result
Action: python_repl_ast
Action Input: df.groupby(['Age', 'Sex']).size().reset_index(name='counts')
Observation:     Age Sex  counts
0    12   F       5
1    12   M       3
2    13   F       3
3    13   M       4
4    14   F       5
5    14   M       7
6    15   F       2
7    15   M       5
8    16   F       2
9    16   M       1
10   17   F       1
11   17   M       2
Thought: I need to save the result to a csv file
Action: python_repl_ast
Action Input: df.groupby(['Age', 'Sex']).size().reset_index(name='counts').to_csv('BigClass_cross.csv', index=False)
Observation: 
Thought: I now know the final answer
Final Answer: The Age×Sex aggregation result is saved to the file BigClass_cross.csv.

> Finished chain.


'The Age×Sex aggregation result is saved to the file BigClass_cross.csv.'

In [39]:
pd.read_csv('BigClass_cross.csv')

,Age,Sex,counts
0,12,F,5
1,12,M,3
2,13,F,3
3,13,M,4
4,14,F,5
5,14,M,7
6,15,F,2
7,15,M,5
8,16,F,2
9,16,M,1


#### Multi CSV Example

In [41]:
agent2 = create_csv_agent(
    llm=ChatOpenAI(temperature=0),
    path=["../../data/BigClass.csv", "BigClass_v2.csv"],
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [42]:
agent2.run("2つのデータセットの違いは？")



> Entering new  chain...
2つのデータセットの違いは、列の数と列の名前です。df1には"Name"、"Age"、"Sex"、"Height_inch"、"Weight_pound"の5つの列があります。一方、df2には"Unnamed: 0"、"Name"、"Age"、"Sex"、"Height_inch"、"Weight_pound"、"Height_cm"、"Weight_kg"、"BMI"の9つの列があります。また、df2には"Unnamed: 0"という列が追加されています。

> Finished chain.


'2つのデータセットの違いは、列の数と列の名前です。df1には"Name"、"Age"、"Sex"、"Height_inch"、"Weight_pound"の5つの列があります。一方、df2には"Unnamed: 0"、"Name"、"Age"、"Sex"、"Height_inch"、"Weight_pound"、"Height_cm"、"Weight_kg"、"BMI"の9つの列があります。また、df2には"Unnamed: 0"という列が追加されています。'

In [45]:
agent2.run("2つのデータセットを縦結合できますか？")



> Entering new  chain...
はい、2つのデータセットを縦結合することができます。pandasの`concat`関数を使用することで実現できます。以下に例を示します。

```python
import pandas as pd

# 2つのデータフレームを作成
df1 = pd.DataFrame({'Name': ['KATIE', 'LOUISE', 'JANE'],
                    'Age': [12, 12, 12],
                    'Sex': ['F', 'F', 'F'],
                    'Height_inch': [59, 61, 55],
                    'Weight_pound': [95, 123, 74]})

df2 = pd.DataFrame({'Name': ['JACLYN', 'LILLIE'],
                    'Age': [12, 12],
                    'Sex': ['F', 'F'],
                    'Height_inch': [66, 52],
                    'Weight_pound': [145, 64]})

# データフレームを縦結合
df_concat = pd.concat([df1, df2], ignore_index=True)

print(df_concat)
```

出力結果:

```
     Name  Age Sex  Height_inch  Weight_pound
0   KATIE   12   F           59            95
1  LOUISE   12   F           61           123
2    JANE   12   F           55            74
3  JACLYN   12   F           66           145
4  LILLIE   12   F           52            64
```

このように、`con

"はい、2つのデータセットを縦結合することができます。pandasの`concat`関数を使用することで実現できます。以下に例を示します。\n\n```python\nimport pandas as pd\n\n# 2つのデータフレームを作成\ndf1 = pd.DataFrame({'Name': ['KATIE', 'LOUISE', 'JANE'],\n                    'Age': [12, 12, 12],\n                    'Sex': ['F', 'F', 'F'],\n                    'Height_inch': [59, 61, 55],\n                    'Weight_pound': [95, 123, 74]})\n\ndf2 = pd.DataFrame({'Name': ['JACLYN', 'LILLIE'],\n                    'Age': [12, 12],\n                    'Sex': ['F', 'F'],\n                    'Height_inch': [66, 52],\n                    'Weight_pound': [145, 64]})\n\n# データフレームを縦結合\ndf_concat = pd.concat([df1, df2], ignore_index=True)\n\nprint(df_concat)\n```\n\n出力結果:\n\n```\n     Name  Age Sex  Height_inch  Weight_pound\n0   KATIE   12   F           59            95\n1  LOUISE   12   F           61           123\n2    JANE   12   F           55            74\n3  JACLYN   12   F           66           145\n4  LILLIE   12   F           52            64\n```\n\nこ